# Learning of the Transmitter Orientation via Gradient Descent

This notebook reproduces the results for the example application "Optimization of transmitter orientation" in the paper [Sionna RT: Differentiable Ray Tracing for Radio Propagation Modeling](https://arxiv.org).

It requires [Sionna](https://github.com/NVlabs/sionna) v0.16 or later.


## Imports and GPU Configuration

In [ ]:
# Set some environment variables
import os
gpu_num = "" # GPU to be used. Use "" to use the CPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Suppress some TF warnings
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    os.system("pip install sionna")
    import sionna

# Configure GPU
import tensorflow as tf
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     try:
#         tf.config.experimental.set_memory_growth(gpus[0], True)
#     except RuntimeError as e:
#         print(e)

# Avoid warnings from TensorFlow
import warnings
tf.get_logger().setLevel('ERROR')
warnings.filterwarnings('ignore')

# Other imports
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from sionna.rt import load_scene, PlanarArray, Transmitter, Receiver, Camera

# Fix the seed for reproducible results
tf.random.set_seed(1)

pre_optimisation_cm = []
post_optimisation_cm = []

## Configure the Scene and Generate Reference Data

In this example, we load the scene "etoile" which is already available in Sionna and then place a single transmitter **with trainable orientation** within the scene.


In [ ]:
# Load the scene
#scene = load_scene(sionna.rt.scene.etoile)
# Load integrated scene
scene = load_scene("/Users/andreamaestri/Desktop/tectwin/GEO/geoloc-rf/scenes/outdoor_linkhall_mockup/IIS_DJI.xml") 
scene.frequency = 3.5e9
# Set the scattering coefficient of the radio material in the scene
# to a non-zero value to enable scattering

In [ ]:
scene._scene_params

In [ ]:
scene.get("elm__3").radio_material = "itu_concrete"

In [ ]:
scene._clear()

In [ ]:
# Configure the transmit array
scene.tx_array = PlanarArray(num_rows=1, num_cols=16,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="tr38901",
                             polarization="V",
                             polarization_model=2)

# Configure the receive array (use to compute the coverage map)
scene.rx_array = PlanarArray(num_rows=1, num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="V",
                             polarization_model=2)

# Create a transmitter and add it to the scene
#transmitter_coordinates = [[-37.47,-46.16,6],[-30.13,-27.22,6],[-30.2714, -28,6],[54.74, -99.,6], [68.12, -82.21, 6], [13.37, -43.53,6],[37.13, -36.03,6]]
#transmitter_coordinates = [[-8.05184,23.06,6.91],[11.39,16.05,0.9185],[31.43, 30.23, 6.93]]
transmitter_coordinates = [np.array([-4.18, 38.12, 16]), np.array([22.96, 16.6, 16]), np.array([43.58, -1.604, 16])]
transmitter_coordinates = [np.array([-4.18, 38.12, 16]), np.array([43.58, -1.604, 16])]
transmitter_coordinates = [np.array([0, 0, 25]), np.array([0, 20, 25])]


tx = Transmitter(f"tx0", position=transmitter_coordinates[0],
                    orientation=tf.Variable([0.0, 0.0, 0.0], tf.float32)) # Trainable orientation

scene.add(tx)
tx = Transmitter(f"tx1", position=transmitter_coordinates[1],
                    orientation=tf.Variable([0.0, 0.0, 0.0], tf.float32)) # Trainable orientation

scene.add(tx)

scene.add(Receiver("rx", position=[30, -30, 8]))
# tx = Transmitter(f"tx2", position=transmitter_coordinates[2],
#                     orientation=tf.Variable([0.0, 0.0, 0.0], tf.float32)) # Trainable orientation

# scene.add(tx)

# Render the scene
# The transmitter is indicated by a blue ball
# cam = Camera("my_cam", position=(0,0,15))
# scene.add(cam)
# cam.look_at([0,0,0])
# scene.render(cam);

In [ ]:
scene.add(Camera("cam", position=[100,-100,130], look_at=[0, 0, 0]))

In [ ]:
scene.render(camera="cam", num_samples=256);

Next, we will compute and show a coverage map for the transmitter `"tx"`. The coverage map corresponds to the average received power into small rectangular regions (or *cells*) of a plane, which, by default, is parallel to the XY plane, and sized such that it covers the entire scene with an elevation of $1.5$ m.

The coverage map is divided into cells of size ``cm_cell_size`` (in meters). The smaller the size of the cells, the more precise the coverage map.

In [ ]:
displacement_vec = [2, 0, 0]
num_displacements = 10
for i in range(num_displacements+1):

    paths = scene.compute_paths(max_depth=5)
    cm = scene.coverage_map(num_samples=10e5,
                            max_depth=5,
                            diffraction=True,
                            cm_center=[0,0,5],
                            cm_orientation=[0,0,0],
                            cm_size=[186,186],
                            cm_cell_size=[1,1])
    scene.render_to_file("cam", f"frame_{i}.png" , coverage_map=cm, paths=paths, num_samples=512)

    # Move TX to next position
    scene.get("rx").position -= displacement_vec